# 문제
- `n(내가 쏠 화살의 수)`와 `info`(상대의 스코어 리스트, 길이는 11이며 10점부터 0점까지 꽂힌 화살의 수)
- 점수 계산은 이런 방식임
    - 같은 점수에 꽂혔다면 더 많이 꽂힌 사람이 그 점수를 가져간다
    - 동점이라면 상대가 점수를 가져간다
    - 둘다 0개라면 아무도 점수를 못 가져간다
- 이때, 내가 얻는 점수를 최대로 하는 화살 배치 배열을 구하라
    - 만약 얻는 점수가 최대인 배열이 여러 개 있다면 작은 점수에 화살을 더 많이 꽂는 배열을 구하라
---
## 풀이
- 나는 DP를 생각했는데, 이전 케이스 -> 다음 케이스를 생각하는 게 쉽지 않았음
- 그래서 풀이를 찾아보니 DFS로 풀더라
0. 어떤 경우에 어떤 상황이 일어날지 모르고, 최종적인 비교는 결국 n발을 다 넣은 상황에서 일어남
    - 따라서 dfs으로 끝까지 간 다음에 거기서 비교를 하고 갱신하는 거임
1. 주어진 배열의 0번째 인덱스부터 살펴봄.
2. 0번째 인덱스의 값에 상관 없이, 선택할 수 있는 케이스는 2가지이다.
    1. 상대의 화살보다 1개 더 넣기
        - 스코어들과 화살의 갯수, 인덱스 + 1, 리스트 변경 -> 다음 재귀
    2. 넣지 않고 지나가기
        - 인덱스 + 1만 해서 다음 재귀
3. 재귀식에 필요한 것은 `상대 스코어`, `내 스코어`, `사용한 화살의 갯수`, `과녁판의 인덱스`, `내 리스트`, `상대 리스트`,`총 화살의 갯수`이다. 
    - `과녁판의 인덱스` : 10점부터 화살을 넣을까 말까 경우의 수를 쫙 펼친다
    - `상대 스코어`, `내 스코어` : 화살을 넣을 때 두 스코어 모두에 변동이 발생한다. 
    - `화살의 갯수` : 위의 2가지 경우에 따라 화살을 몇개 넣을 지 결정됨, 
    - `내 리스트` : 10점부터 인덱스를 늘려가면서 내 경우의 수를 구현해나간다.
        - 이 풀이의 경우 매 재귀에서 기존 점수판을 복사 -> 복사한 점수판 갱신 -> 다음 점수판에 전달 방식을 취함.
    - `상대 리스트`, `총 화살의 갯수`는 인풋이기도 한데, 프로그래머스 코테에서는 이렇게 정의해줘야 인식하더라
4. `cnt > n`이라면 없는 경우의 수를 찾은 거니까 그냥 `return`
5. 4를 제외한 상황에서 모든 인덱스에 대해 돌았다면 `idx > 10` 점수를 계산한다
    - 기존 최고 점수와 같다면, 적은 과녁에 화살이 많이 꽂혀야 하므로 뒤쪽의 인덱스부터 비교해나간다.
        - 만약 기존의 점수판의 작은 인덱스 값이 더 크다면 더 이상 진행할 필요 없음
        - 새로운 점수판의 인덱스 값이 더 크다면 이걸로 갱신함
    - 기존 점수보다 크다면, 남는 화살을 0점 몰아박고 최고 점수와 점수판을 갱신함
        - 0점에 몰아박는 이유는 가장 작은 화살이 많아야 한다는 조건 + 점수가 바뀌면 안되기 때문

In [ ]:
def dfs(a_score, l_score, cnt, idx, board, info, n):
    
    global max_diff, max_board
    
    if cnt > n:
        return
    
    if idx > 10: # 0점까지 끝났다면 계산
        diff = l_score - a_score
        
        if diff == max_diff: # 최대 스코어와 동일하다면, 낮은 점수의 화살이 많아야 함
            for i, j in zip(board[::-1], max_board[::-1]):
                if i < j:
                    break
                
                elif i > j:
                    max_board = board
                    break
        if diff > max_diff:
            board[10] = n - cnt
            max_board = board
            max_diff = diff
        return

    # 상대의 상황과 비교
    now = info[idx]

    # 내가 선택할 수 있는 건 고르지 않거나 상대보다 하나 더
    for i in [0, now + 1]:
        board2 = board.copy()
        
        # score + 10 - idx는 화살들로 얻을 수 있는 점수가 (10 - idx)이기 때문에 그러함
        # 즉 각 인덱스를 돌면서 
        
        # 1. 상대보다 하나 더 쏘기
        if i == now + 1:
            board2[idx] = now + 1
            dfs(a_score, l_score + 10 - idx, cnt + now + 1, idx + 1, board2, info, n)
        
        # 2. 안 쏘기
        else:
            if info[idx] > 0: # 화살이 있다면
                dfs(a_score + (10 -idx), l_score, cnt, idx + 1, board2, info, n)
            else:
                dfs(a_score, l_score, cnt, idx + 1, board2, info, n)
        

def solution(n, info):
    global max_diff, max_board
    answer = []
    max_diff = 0
    max_board = []
    board = [0] * 11 # 내 스코어판

    dfs(0, 0, 0, 0, board, info, n)
    
    return max_board if max_board else [-1]